In [16]:
from utils.__init__  import get_transaction_details

# Example usage
# txn_intent_hash = "txid_rdx1e7yx9ess0ulzpzhepdh9zaffcet0f70x93xdk7v9t83qrn738vfqc0es0w"  # Remove liquidity
# txn_intent_hash = "txid_rdx12t6cs0n65wex7s9c52kpy4943xhx4uap0p9y6c08uf5m43gwch7skdk8ge"  # Add liquidity for c9 lsulp/xrd pool
txn_intent_hash = "txid_rdx16ve43rsp4nlw45z4jmysqvskphnr5cakgz0jkftekrfhrnpxx2fqumvq88"  # Add liquidity for c9 xrd/xusdc shape liquidity pool
txn_details = get_transaction_details(txn_intent_hash)

txn_details.json()
state_version = txn_details.json()['transaction']['state_version']   

# Call the function with the state_version we got earlier
from utils.__init__ import get_transaction_stream


response = get_transaction_stream(state_version)
parsed_response = response.json()
transaction_data = [x for x in parsed_response.get('items') if x.get('intent_hash') == txn_intent_hash]
transaction_data

[{'transaction_status': 'CommittedSuccess',
  'state_version': 286388055,
  'epoch': 203033,
  'round': 449,
  'round_timestamp': '2025-05-13T16:00:30.529Z',
  'payload_hash': 'notarizedtransaction_rdx1zjcqtglu9q639jjnr3xrsyl2vc5382c5xpz6vptmadc6muvrzmfsgrjdp0',
  'intent_hash': 'txid_rdx16ve43rsp4nlw45z4jmysqvskphnr5cakgz0jkftekrfhrnpxx2fqumvq88',
  'fee_paid': '0.86948033275',
  'confirmed_at': '2025-05-13T16:00:30.529Z',
  'receipt': {'status': 'CommittedSuccess',
   'detailed_events': [{'identifier': {'package': 'package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx000538436477xxxxxxxxxresrce',
      'blueprint': 'FungibleVault',
      'event': 'LockFeeEvent'},
     'payload_type_definition': {'local_type_id': 22,
      'entity': 'package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx000538436477xxxxxxxxxresrce',
      'schema_hash_hex': '462a3fea283117aab2b01c297812bdc0fa9060b29eb5e68b847f361bc1201933',
      'kind': 'SchemaLocal'},
     'emitter': {'method_emitter': {'entity': 'internal_vault_rdx1tqugz497eal

In [17]:
import pandas as pd


events = transaction_data[0]['receipt']['detailed_events']
# Extract data from events
events

# Initialize an empty list to store event data
event_data = []

# Process each event and extract relevant information
for event in events:
    # Initialize a dictionary for this event
    event_dict = {}
    
    # Extract identifier information
    if 'identifier' in event:
        event_dict['name'] = event['identifier'].get('event', '')
        event_dict['blueprint'] = event['identifier'].get('blueprint', '')
        event_dict['package'] = event['identifier'].get('package', '')
    
    # Extract emitter information
    if 'emitter' in event:
        event_dict['emitter_type'] = event['emitter'].get('type', '')
        
        # Handle different emitter structures
        if 'method_emitter' in event['emitter'] and 'entity' in event['emitter']['method_emitter']:
            entity = event['emitter']['method_emitter']['entity']
            if isinstance(entity, str):
                event_dict['entity_address'] = entity
            elif isinstance(entity, dict):
                event_dict['entity_address'] = entity.get('entity_address', '')
        
        if 'global_emitter' in event['emitter']:
            event_dict['global_emitter'] = event['emitter']['global_emitter']
    
    # Extract payload information
    if 'payload' in event and 'programmatic_json' in event['payload']:
        payload = event['payload']['programmatic_json']
        event_dict['data_kind'] = payload.get('kind', '')
        
        # Extract fields from payload based on structure
        if payload.get('kind') == 'Tuple' and 'fields' in payload:
            for field in payload['fields']:
                if 'field_name' in field and 'value' in field:
                    event_dict[field['field_name']] = field['value']
        
        # Handle Enum type payloads
        elif payload.get('kind') == 'Enum' and 'fields' in payload:
            event_dict['variant_name'] = payload.get('variant_name', '')
            
            for field in payload['fields']:
                if isinstance(field, dict):
                    if 'field_name' in field and 'value' in field:
                        event_dict[field['field_name']] = field['value']
                    elif 'type_name' in field and 'value' in field:
                        event_dict[field['type_name']] = field['value']
    
    # Add the event dictionary to our list
    event_data.append(event_dict)

# Create a pandas DataFrame from the event data
events_df = pd.DataFrame(event_data)

# Display the DataFrame
events_df

,name,blueprint,package,emitter_type,entity_address,global_emitter,data_kind,amount,variant_name,ResourceAddress,liquidity_receipt_id,amount_change_x,amount_change_y,amount_after_x,amount_after_y,price_after
0,LockFeeEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tqugz497ealrra38wsj2np7xnz8...,account_rdx16xqpkw4lujmuuyt6hdqgjutdlu0d7kqlqc...,Tuple,1.0775376581885,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,WithdrawEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tqugz497ealrra38wsj2np7xnz8...,account_rdx16xqpkw4lujmuuyt6hdqgjutdlu0d7kqlqc...,Tuple,16.44707901303071768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,WithdrawEvent,Account,package_rdx1pkgxxxxxxxxxaccntxxxxxxxxxx0009296...,EntityMethod,account_rdx16xqpkw4lujmuuyt6hdqgjutdlu0d7kqlqc...,account_rdx16xqpkw4lujmuuyt6hdqgjutdlu0d7kqlqc...,Enum,NaN,Fungible,resource_rdx1tknxxxxxxxxxradxrdxxxxxxxxx009923...,NaN,NaN,NaN,NaN,NaN,NaN
3,WithdrawEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tp9qqknnk54gd6l5phjgt9q9jug...,account_rdx16xqpkw4lujmuuyt6hdqgjutdlu0d7kqlqc...,Tuple,0.8128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,WithdrawEvent,Account,package_rdx1pkgxxxxxxxxxaccntxxxxxxxxxx0009296...,EntityMethod,account_rdx16xqpkw4lujmuuyt6hdqgjutdlu0d7kqlqc...,account_rdx16xqpkw4lujmuuyt6hdqgjutdlu0d7kqlqc...,Enum,NaN,Fungible,resource_rdx1t4upr78guuapv5ept7d7ptekk9mqhy605...,NaN,NaN,NaN,NaN,NaN,NaN
5,MintNonFungibleResourceEvent,NonFungibleResourceManager,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,resource_rdx1ntzhjg985wgpkhda9f9q05xqdj8xuggfw...,resource_rdx1ntzhjg985wgpkhda9f9q05xqdj8xuggfw...,Tuple,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MintLiquidityReceiptEvent,QuantaSwap,package_rdx1p4r9rkp0cq67wmlve544zgy0l45mswn6h7...,EntityMethod,component_rdx1cr6lxkr83gzhmyg4uxg49wkug5s4wwc3...,component_rdx1cr6lxkr83gzhmyg4uxg49wkug5s4wwc3...,Tuple,NaN,NaN,NaN,{8988cc0aa2312d07-cb95b730330bf073-d6401f97c54...,NaN,NaN,NaN,NaN,NaN
7,DepositEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1trgadcvlrmyrtv0zhh339ahhu4j...,component_rdx1cr6lxkr83gzhmyg4uxg49wkug5s4wwc3...,Tuple,10.215216109014984394,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,DepositEvent,FungibleVault,package_rdx1pkgxxxxxxxxxresrcexxxxxxxxx0005384...,EntityMethod,internal_vault_rdx1tr2nd37d7rcpusgjz2qqktkq8d5...,component_rdx1cr6lxkr83gzhmyg4uxg49wkug5s4wwc3...,Tuple,0.8128,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,AddLiquidityEvent,QuantaSwap,package_rdx1p4r9rkp0cq67wmlve544zgy0l45mswn6h7...,EntityMethod,component_rdx1cr6lxkr83gzhmyg4uxg49wkug5s4wwc3...,component_rdx1cr6lxkr83gzhmyg4uxg49wkug5s4wwc3...,Tuple,NaN,NaN,NaN,{8988cc0aa2312d07-cb95b730330bf073-d6401f97c54...,10.215216109014984394,0.8128,NaN,NaN,NaN


In [18]:
# Filter the DataFrame for 'AddLiquidityEvent' and display 'amount_change_x' and 'amount_change_y'
events_df[events_df['name'] == 'AddLiquidityEvent'][['name', 'amount_change_x', 'amount_change_y', 'global_emitter', 'liquidity_receipt_id']]


,name,amount_change_x,amount_change_y,global_emitter,liquidity_receipt_id
9,AddLiquidityEvent,10.215216109014984394,0.8128,component_rdx1cr6lxkr83gzhmyg4uxg49wkug5s4wwc3...,{8988cc0aa2312d07-cb95b730330bf073-d6401f97c54...
